## Example - The Moon Voyage

I've waffled on what example to use for ``wig``/``MCNP`` for a while.  All the nuclear stuff that isn't boring is, in general, classified or difficult.  So, I went for something interesting, but completely unrealistic: Jules Verne's *The Moon Voyage*.  In the book, Mr. Verne describes launching a bullet shaped projectile from Tampa to the moon.  I think it'd be interesting to see, among other reasons why the inhabitants would surely be unable to survive, how much radiation dose they would get on their trip.  ``MCNP`` is great for neutral particles, and not so great for continuum interactions, so we'll ignore the cosmic protons here (which are admittedly a huge part of solar radiation), but we will take into account solar photons with their spectrum.

The first thing we have to do to start the simulation is to import ``wig``.  This is reasonably simple: ``wig`` has a main class, called ``wig.wig`` and then seven other classes, imported below, named (hopefully) self-explanatorially. We should also import ``numpy``, because I can't imagine life without it.  And finally, we import some functions that convert imperial units to centimeters, because Mr. Verne didn't know any better than to use them.

In [1]:
from wig import wig, geo, cell, matl, phys, source, tally
import numpy as np
# I know this is bad form, but I want to use feet(9) instead of units.feet(9), it's just easier
from units import *

Now, we'll have to start our scene.  We'll use ``MCNP6`` to make use of the new photon libraries (and photoneutron interactions), so to run this, you'll have to have ``mcnp6`` on your path.

In [2]:
scene = wig.wig(filename='moon_voyage_projectile', flavor='6',
                comment='''Solar photons incident on a bullet shaped aluminum
                           projectile from Jules Vernes The Moon Voyage''',
                render=True)

Next, we'll make the geometry.  Note that ``MCNP`` splits geometry and cells into two modules - you'll have to first define geometries and then combine them into cells.  ``wig`` attempts to make this more semantic, but it definitely could still use some help.  Note also that ``wig`` does not have to be written in order.  You could write materials, then physics, then sources, then geometry and cells; however, the geometry and materials **must** be defined before the cells - but that's just basic programming, you have to define variables before you use them.

So below, we'll generate a reasonably complex geometry, as Verne describes the projectile as:

> ...a cylindro-conical projectile... The projectile outside was nine feet wide and twelve feet high. ...under the wadding were four thick lenticular footlights, two let into the circular wall of the projectile, the third in its lower part...

So now let's get to modeling. First, we need the outer surfaces of the bullet.  We'll create a cylinder for the base, a cone for the top, and we'll use a sphere to soften the connection between the cylinder and cone.

In [3]:
# the outside cylinder, centered around the origin - which means the base has to go down halfway
# also, we must convert everything from imperial units to centimeters
# bullet outer radius in centimeters
bullet_or = 4.5 * 12. * 2.540
# bullet cylindrical length in centimeters
bullet_h = 10. * 12. * 2.540
# bullet offset from origin to place the floor at the origin
bullet_dz = -19. * 2.540
# bullet point height is 12 feet, converted to centimeters
bullet_point_z = 16. * 12. * 2.540
bocyl = geo.geo().rcc(c=(0., 0., bullet_dz), r=bullet_or, lz=bullet_h, id='bullet_outer_cyl')
# the sphere to soften the curve - must hit tangent at the top of the cylinder
bosph = geo.geo().sph(c=(0., 0., bullet_dz + bullet_h), r=bullet_or, id='bullet_outer_fillet')
# Now the cone going from a point on the sphere, tangent to the sphere, up to a point
# find the place on the sphere where it is 45deg tangent
dz = bullet_point_z - bullet_h
dz1 = ((dz * dz) - (bullet_or * bullet_or)) / dz
dz2 = dz - dz1
ocone_bottom_h = dz2
ocone_bottom_r = np.sqrt((bullet_or * bullet_or) - (dz2 * dz2))
# Now find the radius at that point
bocone = geo.geo().cone(c=(0., 0., bullet_dz + bullet_h + ocone_bottom_h), r1=ocone_bottom_r, r2=0.1,
                        lz=dz1, id='bullet_outer_point')

Now, we can add the hollow section, which the book claims is 12" thick in most places.  We should make the bottom 18" thick, because it was noted to be thicker at the bottom.

In [4]:
bullet_ir = feet(3.5) #3.5 * 12. * 2.540
bullet_ih = feet(8.5) #8.5 * 12. * 2.540
bullet_idz = -inches(1) #-1. * 2.540
bicyl = geo.geo().rcc(c=(0., 0., bullet_idz), r=bullet_ir, lz=bullet_ih, id='bullet_inner_cyl')
# the sphere to soften the curve - must hit tangent at the top of the cylinder
bisph = geo.geo().sph(c=(0., 0., bullet_dz + bullet_h), r=bullet_ir, id='bullet_inner_fillet')
# Now the cone going from a point on the sphere, tangent to the sphere, up to a point
dz = bullet_point_z - bullet_h - feet(1) # (12. * 2.540)
dz1 = ((dz * dz) - (bullet_ir * bullet_ir)) / dz
dz2 = dz - dz1
icone_bottom_h = dz2
icone_bottom_r = np.sqrt((bullet_ir * bullet_ir) - (dz2 * dz2))
bicone = geo.geo().cone(c=(0., 0., bullet_idz + bullet_ih + icone_bottom_h), r1=icone_bottom_r, r2=0.1,
                        lz=dz1, id='bullet_inner_point')

There were at least three portholes, each with a brass surround and glass.

In [5]:
phole1 = geo.geo().rcc(c=(0., 0., - inches(19)), r=inches(5.5), lz=inches(20), id='porthole1')
pholebrass1 = geo.geo().rcc(c=(0., 0., - inches(19)), r=inches(6), lz=inches(20),
                            id='portholebrass1')
phole2 = geo.geo().rcc(c=(-feet(4.5), 0., inches(40)), r=inches(5.5), lx=feet(4.5), id='porthole2')
pholebrass2 = geo.geo().rcc(c=(-feet(4.5), 0., inches(40)), r=inches(6), lx=feet(4.5),
                            id='portholebrass2')
phole3 = geo.geo().rcc(c=(0., -feet(4.5), inches(40)), r=inches(5.5), ly=feet(4.5), id='porthole3')
pholebrass3 = geo.geo().rcc(c=(0., -feet(4.5), inches(40)), r=inches(6), ly=feet(4.5),
                            id='portholebrass3')

The book states that there is a plank of wood holding the room which the inhabitants live in.  This is hermetically sealed around the bottom of the inside of the cylinder.

In [6]:
woodcyl = geo.geo().rcc(c=(0., 0., -inches(1)), r=bullet_ir, lz=inches(1), id='wood_disk')

Now that we have those, we can add an enclosing universe, and add all the geometry to the model

In [7]:
uni = geo.geo().sph(c=(0., 0., 0.), r=feet(15), id='universe')
scene.geo([bocyl, bosph, bocone, bicyl, bisph, bicone,
           woodcyl,
           phole1, phole2, phole3,
           pholebrass1, pholebrass2, pholebrass3,
           uni])

Next, we need to define some materials for this model.  All of these come from the PNNL Compendium.

In [8]:
# air for the insides of the cabin
air = matl.matl(rho=0.001205, id='air', mass_perc = [('C', 0.000124), ('N-14', 0.755268), ('O-16', 0.231781),
                                                     ('Ar', 0.012827)])
# aluminum - note that aluminum alloys were terrible in Jules Verne's time, so I'll just
# assume that its pure
aluminum = matl.matl(rho=2.6989, id='aluminum', color='#888888', atom_perc = [('Al-27', 1.0)])
# The only wood that PNNL gives us is southern pine - but the book is set in Florida,
# which is pretty southern, I guess
wood = matl.matl(rho=0.640, id='wood', color='#966F33',
                 atom_perc = [('H-1', 0.462423), ('C', 0.323389), ('N-14', 0.002773),
                              ('O-16', 0.208779), ('Mg', 0.000639), ('S', 0.001211),
                              ('K', 0.000397), ('Ca', 0.000388)])
# Plate glass for the windows - who knows if they can be casted that thick
glass = matl.matl(rho=2.4, id='glass', color='#7299c6', alpha=0.4,
                  atom_perc = [('O-16', 0.603858), ('Na-23', 0.088145), ('Si', 0.251791),
                               ('Ca', 0.056205)])
# Brass for the surrounds on the outside of the windows
brass = matl.matl(rho=1.0, id='brass', color='#b5a642',
                  atom_perc=[('Fe', 0.001002), ('Cu', 0.674918), ('Zn', 0.320956),
                             ('Sn', 0.001451), ('Pb', 0.001673)])
void = matl.matl(rho=0.0, id='void', mass_perc = [])
scene.matl([air, aluminum, wood, glass, brass])

Now, to make the cells using the geometry and the materials.

In [9]:
%%capture
bullet_cell = cell.cell(bocyl + bosph + bocone - bicyl - bisph - bicone - 
                        pholebrass1 - pholebrass2 - pholebrass3, aluminum)
wood_cell = cell.cell(woodcyl - pholebrass1, wood)
window_1_b = cell.cell(pholebrass1 - phole1 - bicyl, brass)
window_1 = cell.cell(phole1 - bicyl, glass)
window_2_b = cell.cell(pholebrass2 - phole2 - bicyl, brass)
window_2 = cell.cell(phole2 - bicyl, glass)
window_3_b = cell.cell(pholebrass3 - phole3 - bicyl, brass)
window_3 = cell.cell(phole3 - bicyl, glass)
air_cell = cell.cell(bicyl + bisph + bicone - woodcyl, air, show=False)
uni_cell = cell.cell(uni - bocyl - bosph - bocone, void, show=False)
scene.cell([bullet_cell, wood_cell,
            window_1, window_2, window_3,
            window_1_b, window_2_b, window_3_b,
            air_cell, uni_cell])

Now that we have the geometry built, we need some details about the physics.  We're only going to take into account photons for now, and their energies need to span a huge range - we'll span the entire range of the solar irradiance dataset from Wehrli 1985, which is $199.5\mathrm{nm}$ to $10075.0\mathrm{nm}$ (or $.123\mathrm{eV}$  to $6.2\mathrm{eV}$).

In [10]:
# Add some typical neutron physics
physics = phys.phys(particles='p', nps=1E8, maxE=6.2E-6, minE=0.123E-6)
scene.phys(physics)

Now, we'll add a source.  The projectile is far enough from the sun to assume that the sun is a plane source.  We'll add a source covering the projectile to the $-x$ direction in our model.  The spectrum is taken from the ASTM E490-0 AM0 old Spectrum from NREL and Wehrli.  I've downloaded a text file from NREL and will use it to plot the spectrum below. Then, we'll have to convert this to flux instead of irradiance using the formula $$F\left(\lambda\right) = \Phi E \frac{1}{\Delta \lambda}$$ and by solving for $\Phi$ $$\Phi = F\left(\lambda\right)\frac{\Delta\lambda}{E}$$ We're using a couple libraries I maintain for data analysis and plotting called [``pym``](https://alexhagen.github.io/pym) and [``pyg``](http://alexhagen.github.io/pyg), respectively.

In [11]:
from pym import func as pym
from pyg import twod as pyg2d
import shutil
# Wehrli gives irradiance in W/m^2/nm
wavelength, irradiance, _ = \
    np.loadtxt(scene.original_directory + '/wehrli85.txt', unpack=True, skiprows=2)
h = 4.135668E-15 # planck's constant in eV s
c = 2.998E+17 #speed of light in nm/s
E = h * (c / np.array(wavelength))
# Conversion to flux (and converting eV to J)
Phi = (irradiance / (E * 1.6E-19)) * wavelength
flux_spectrum = pym.curve(E, Phi, name='solar_flux_spectrum')
plot = flux_spectrum.plot(linestyle='-')
plot.xlabel(r'Photon Energy ($E$) [$\mathrm{eV}$]')
plot.ylabel(r'Flux ($\Phi$) [$\mathrm{\frac{\text{photon}}{m^{2} \cdot s}}$]')
plot.markers_off()
plot.export('solar_spectrum', ratio='silver')
shutil.copy('solar_spectrum.svg', scene.original_directory + '/solar_spectrum.svg')
plot.show('The solar spectrum from Wehrli 1985 in energy units', label='solspec')

In [12]:
# add the source
sun = source.source(particle='p', pos=(0., -feet(8), feet(6)), id='sun',
                    shape='disk', direction='+y', radius=feet(12), 
                    spectrum=[E, Phi])
scene.source([sun])

And let's check a tally or two.  I think we'd like to have a flux tally inside of the chamber, and also a heating tally in the bullet cell. These commands should be pretty semantic

In [13]:
# add some tallys
mesh_tally = tally.tally(comment='compartment_mesh', particle='p')\
    .mesh_tally(xmin=-feet(4.5), xmax=feet(4.5), i=9*12,
                ymin=-feet(4.5), ymax=feet(4.5), j=9*12,
                zmin=0.0, zmax=feet(10), k=10*12)
energy_tally = tally.tally(comment='aluminum_heating', particle='p', energy=[0.123E-6, 6.2E-6])\
    .energy_tally(tally_cell=bullet_cell)

scene.tally([energy_tally, mesh_tally])

Now, we can run it.  We want to do a rendering of the geometry, so we'll look at that first.

In [14]:
scene2 = scene.bscene
scene2.cutaway(c=(100., 100., feet(8)), l=(200., 200., feet(20)))
scene2.run(res=[1080, 1920], samples=100, 
           c=(0., 0., feet(6)), camera_location=(700., 700., 375.))

In [15]:
from pyg import three2twod as pyg32d

shutil.copy('brender_01.png', scene.original_directory + '/moon_voyage.png')
plot = pyg32d.ann_im(scene.original_directory + '/moon_voyage.png')
plot.add_data_pointer(0., -feet(8), feet(6) + feet(12),
                      string=r'Sun (disk photon source)',
                      place=(-200., 200.))
plot.add_legend_entry(color='#888888', name='Aluminum')\
    .add_legend_entry(color='#b5a642', name='Brass')\
    .add_legend_entry(color='#7299c6', alpha=0.4, name='Glass')\
    .add_legend_entry(color='#966F33', name='Wood')
plot.legend(loc=2)
plot.export('moon_voyage', force=True, ratio='invgolden')
shutil.copy('moon_voyage.svg', scene.original_directory + '/moon_voyage.svg')
plot.show('Depiction of the Moon Voyage\'s cylindo-conical projectile',
          label='moonvoyageprojectile')

And finally, we can run it.  I'll check back in in a week or so with the results!

In [16]:
%%capture
#scene.run(render=False)